In [191]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import re
import json


caps = "([A-Z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + caps + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(caps + "[.]" + caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(caps + "[.]" + caps + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + caps + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [192]:
def get_response_url(partial_url):
    full_url = 'https://vng.nl{}'.format(partial_url)
    r = requests.get(full_url)
    return r.url

def get_soup_from_partial_url(partial_url):
    full_url = 'https://vng.nl{}'.format(partial_url)
    r = requests.get(full_url)
    return BeautifulSoup(r.content, "html5lib")

def get_news_article_urls_from_page(soup):
    urls = []
    
    for a_tag in soup.find_all('a', href=True):

        if '/nieuws/' in a_tag['href']:
            urls.append(a_tag['href'])
            
    return urls

def next_page_soup(soup):    
    next_page_button = soup.find('li', {'class' : 'pager-next'})
    
    if next_page_button:
        return get_soup_from_partial_url(next_page_button.a['href'])
    else:
        return None

    
def read_news_content_from_url(url):
    page_soup = get_soup_from_partial_url(url)
    article = page_soup.find('div', {'about' : url})
    if article:
        content = article.find('div', {'class' : 'content'})
        sentences = split_into_sentences(content.get_text())
    else:
        sentences = []
    
    for sentence in sentences:
        if 'vng-dossier' in sentence.lower():
            sentences.remove(sentence)
    
    return ' '.join(sentences)

def read_label_from_url(url):
    splitted = url.split('/')
    return splitted[2]

In [193]:
urls = []

current_soup = get_soup_from_partial_url('/nieuws')

while current_soup:
    current_urls = get_news_article_urls_from_page(current_soup)
    
    for url in current_urls:
        urls.append(url)
    
    current_soup = next_page_soup(current_soup)

KeyboardInterrupt: 

In [ ]:
result = []

for url in urls:
    print(url)
    
    content = read_news_content_from_url(url)
    response_url = get_response_url(url)

    main = response_url.split('/')[4]
    sub = response_url.split('/')[5]
    
    result.append({'main_topic' : main, 'sub_topic' : sub, 'content' : content})
    

with open('../data_resources/topics/vng_training.json', 'w') as outfile:
    json.dump(result, outfile)